In [1]:
# ==============================================================================
# GPR
# ==============================================================================
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product

# --- 1) Path setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()


NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# --- 2) Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,
    EWMA_CORR_SPEC,
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
# Model import (GPR)
from src.models.GPR import ForecastModel

# --- 3) Config ---
USE_DYNAMIC_FI_PIPELINE = True  # False = standard setup (I & II)
SEED = 42

MODEL_NAME = "gpr_dynamic_fi" if USE_DYNAMIC_FI_PIPELINE else "gpr_without_target"

outputs_for_model(MODEL_NAME)
print(f"--- Starting tuning for: {MODEL_NAME} ---")

# --- 4) Load data ---
y = load_target()
X_ifo = load_ifo_features()

# Align indices
idx_common = y.index.intersection(X_ifo.index)
y = y.loc[idx_common]
X_ifo = X_ifo.loc[idx_common]

X_full_lagged = None
rolling_imp = None
y_fi = None

if USE_DYNAMIC_FI_PIPELINE:
    FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp = load_rolling_importance(base_dir=FI_BASE_DIR)

        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp = rolling_imp.loc[idx_fi]
        print(f"Dynamic FI mode: loaded {X_full_lagged.shape[1]} features.")
    except FileNotFoundError:
        print("ERROR: Dynamic FI artifacts not found.")
        sys.exit(1)
else:
    print(f"Full FE mode (Setup I/II): {X_ifo.shape[1]} base features.")

# --- 5) Config defaults ---
def get_thesis_cfg() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    cfg.policy_window = 24
    cfg.policy_decay = 0.97
    cfg.selection_mode = "decayed_best"
    return cfg


cfg_obj = get_thesis_cfg()

# --- 6) Grid definition ---

def build_grid_full_fe():
    """Setup I (ifo) and Setup II (ifo + target blocks)."""

    # A) FE & DR
    lag_candidates = [tuple(range(7))]

    corr_opts = [
        {"corr_spec": dict(DEFAULT_CORR_SPEC)},
        {"corr_spec": dict(EWMA_CORR_SPEC)},
    ]

    k1_opts = [700]
    red_opts = [0.9, 1.0]

    dr_opts = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_kmax": 30, "pca_var_target": 0.99},
        {"dr_method": "pls", "pls_components": 30},
    ]

    # B) Setup II (target blocks)
    block_opts = [
        None,  # Setup I
        # ["AR1", "Chronos", "TSFresh"]  # Setup II b
    ]

    # C) Weights
    weight_opts = [
        {"sample_weight_decay": None},
        # GPR supports sample weights via alpha modulation (see src/models/GPR.py)
        {"sample_weight_decay": 0.99},
    ]

    # D) GPR hyperparameters
    kernel_list = ["rbf", "matern"]
    ard_list = [True]       # ARD always on here
    nu_list = [1.5, 2.5]    # Only used for Matern

    n_restarts_list = [2, 3, 5, 10, 20]

    noise_alpha_list = [1e-4]          # Small jitter for stability
    length_scale_init_list = [1.0]

    grid = []

    # 1) FE loop
    for lags, corr, k1, red, dr in product(lag_candidates, corr_opts, k1_opts, red_opts, dr_opts):
        base_fe = {
            "lag_candidates": lags,
            "k1_topk": k1,
            "redundancy_param": red,
            **dr,
            **corr,
        }

        # 2) Blocks & weights
        for blocks, weights in product(block_opts, weight_opts):

            # 3) Model hyperparameters
            for k, ard, nu, restarts, noise, ls_init in product(
                kernel_list, ard_list, nu_list, n_restarts_list, noise_alpha_list, length_scale_init_list
            ):
                # Skip redundant RBF nu combinations
                if k == "rbf" and nu != 1.5:
                    continue

                hp = {
                    **base_fe,
                    "target_block_set": blocks,
                    **weights,
                    "kernel": k,
                    "ard": ard,
                    "nu": nu,
                    "n_restarts_optimizer": restarts,
                    "noise_alpha": noise,
                    "length_scale_init": ls_init,
                    "normalize_y": True,
                    "seed": SEED,
                }
                grid.append(hp)

    return grid


def build_grid_dynamic_fi():
    """Setup III: Dynamic feature importance via strict Top-N."""

    n_features_list = [50]  # Fixed number of features

    weight_opts = [{"sample_weight_decay": None}, {"sample_weight_decay": 0.99}]

    # GPR params
    kernel_list = ["rbf", "matern"]
    ard_list = [True]
    nu_list = [1.5, 2.5]
    n_restarts_list = [2, 10]
    noise_alpha_list = [1e-4]
    length_scale_init_list = [1.0]

    grid = []
    for n_feat, w in product(n_features_list, weight_opts):
        for k, ard, nu, restarts, noise, ls_init in product(
            kernel_list, ard_list, nu_list, n_restarts_list, noise_alpha_list, length_scale_init_list
        ):
            if k == "rbf" and nu != 1.5:
                continue

            hp = {
                "n_features_to_use": n_feat,
                **w,
                "kernel": k,
                "ard": ard,
                "nu": nu,
                "n_restarts_optimizer": restarts,
                "noise_alpha": noise,
                "length_scale_init": ls_init,
                "normalize_y": True,
                "seed": SEED,
            }
            grid.append(hp)
    return grid


# --- 7) Run ---

if USE_DYNAMIC_FI_PIPELINE:
    grid = build_grid_dynamic_fi()
    print(f"Dynamic FI grid size (Setup III): {len(grid)} configurations.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,  # dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp,
        keep_top_k_final=5,
        min_survivors_per_block=5,
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,  # dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp,
    )

else:
    grid = build_grid_full_fe()
    print(f"Full FE grid size (Setup I & II): {len(grid)} configurations.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
        keep_top_k_final=5,
        min_survivors_per_block=5,
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
    )

print("\nTuning finished.")


--- Starte Tuning für: gpr_dynamic_fi ---
INFO in load_ifo_features: Renaming columns to ensure validity.
Dynamic FI Modus: 4320 Features geladen.
Dynamic FI Grid Größe (Setup III): 12 Konfigurationen.
[Stage A] Using DYNAMIC FI (Gleis 3) pipeline.
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=12
  - Config 1/12
    · Month 5/20 processed | running...RMSE=1.6460
    · Month 10/20 processed | running...RMSE=1.3145
    · Month 15/20 processed | running...RMSE=1.2341
    · Month 20/20 processed | running...RMSE=1.1015
  - Config 2/12
    · Month 5/20 processed | running...RMSE=1.6460
    · Month 10/20 processed | running...RMSE=1.3145
    · Month 15/20 processed | running...RMSE=1.2341
    · Month 20/20 processed | running...RMSE=1.1015
  - Config 3/12
    · Month 5/20 processed | running...RMSE=1.6460
    · Month 10/20 processed | running...RMSE=1.3145
    · Month 15/20 processed | running...RMSE=1.2341
    · Month 20/20 processed | running...RMSE=1.1015
  - Config 4/12
    · M

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 9 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 18 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 41 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find

[Stage B] Month origin t=241 | evaluating 5 configs | active=5


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 6 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 28 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 29 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find

[Stage B] Month origin t=242 | evaluating 5 configs | active=1


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 20 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 3 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 34 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find

[Stage B] Month origin t=405 | evaluating 5 configs | active=5


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 3 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 5 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a

[Stage B] done → /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/stageB/gpr_dynamic_fi/monthly
[Stage B] summary.csv & summary_active.txt geschrieben.

Tuning abgeschlossen.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 12 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 49 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
